In [1]:
## import warnings
import time
import math
import os
import glob
from pyhere import here
from datetime import date

import numpy as np
import pandas as pd
import geopandas
import pickle

import pyarrow
import itertools
import multiprocessing
import p_tqdm

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, LeaveOneGroupOut, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from scipy.stats import spearmanr,  pearsonr

In [2]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 1, 2])
g = np.array([1, 1, 2, 2])

logo = LeaveOneGroupOut()
ridge = Ridge()

parameters = {'alpha': np.logspace(-5, 5, base = 10, num = 11)}

In [3]:
ridge_logo_reg = GridSearchCV(ridge, parameters, scoring = 'r2', cv = logo)
ridge_logo_reg.fit(X, y, groups = g)
best_logo_model = ridge_logo_reg.best_estimator_

In [4]:
ridge_logo_reg.best_estimator_.score(X, y)

0.09795918367346945

In [5]:
logo_val_predictions = cross_val_predict(
    best_logo_model, X = X, y = y, groups = g,  cv = logo)   

array([1.40384615, 1.44230769, 1.55769231, 1.59615385])

In [6]:
r2_score(y, logo_val_predictions)

0.05177514792899407

In [7]:
best_logo_model.fit(X, y, g)

Ridge(alpha=100.0)

In [8]:
best_logo_model.score(X, y, g)

0.12774970273483977

In [9]:
pd.DataFrame(ridge_logo_reg.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001032,3.201962e-04,0.000388,8.451939e-05,0.00001,{'alpha': 1e-05},-14.999920,-14.999920,-14.999920,2.808667e-15,11
1,0.000665,2.737045e-04,0.000269,1.513958e-05,0.0001,{'alpha': 0.0001},-14.999200,-14.999200,-14.999200,2.808667e-15,10
2,0.000393,6.914139e-06,0.000252,1.072884e-06,0.001,{'alpha': 0.001},-14.992003,-14.992003,-14.992003,1.256074e-15,9
3,0.000416,9.584427e-05,0.000220,1.180172e-05,0.01,{'alpha': 0.01},-14.920305,-14.920305,-14.920305,0.000000e+00,8
4,0.000327,1.072884e-05,0.000206,2.741814e-06,0.1,{'alpha': 0.1},-14.229625,-14.229625,-14.229625,4.528839e-15,7
5,0.000315,2.384186e-07,0.000205,9.536743e-07,1.0,{'alpha': 1.0},-9.280000,-9.280000,-9.280000,2.808667e-15,6
6,0.000321,5.483627e-06,0.000214,1.060963e-05,10.0,{'alpha': 10.0},-0.816327,-0.816327,-0.816327,5.551115e-16,5
7,0.000359,4.887581e-06,0.000224,1.525879e-05,100.0,{'alpha': 100.0},0.051775,0.051775,0.051775,2.220446e-16,1
8,0.000316,1.311302e-06,0.000206,8.344650e-07,1000.0,{'alpha': 1000.0},0.007698,0.007698,0.007698,0.000000e+00,2
9,0.000319,1.430511e-06,0.000223,1.728535e-05,10000.0,{'alpha': 10000.0},0.000797,0.000797,0.000797,0.000000e+00,3
